## Entendendo como extrair informações sobre repositórios no github

[Search](https://developer.github.com/v3/search/)
[Searching for repositories](https://help.github.com/en/articles/searching-for-repositories#search-by-repository-name-description-or-contents-of-the-readme-file)

### Lista de qualifiers:  https://help.github.com/en/articles/searching-code
### Documentação para search: https://developer.github.com/v3/search/

In [27]:
import requests
import pandas as pd

Montando url de busca para os repositórios com maior quantidade de estrelas.
Usando como consulta stars:>1 para indicar que são todos os repositorios acima de 1 estrela, o que não tem impacto negativo para a busca pois existem certa de 3.305.358 repositórios e só vamos analisar 2.500.

In [4]:
q = 'q=stars:>1'
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?'
url_final = url_base+q+sort
url_final

'https://api.github.com/search/repositories?q=stars:>1&sort=stars&order=desc'

In [5]:
def percorrendo_paginas(url):
    
    results = requests.get(url_final)
    
    data = dict(results.json())
    data.keys()
    
    header = dict(results.links)
    header.keys()

    data.get('total_count')

    itens_por_pagina = len(data.get('items'))
    itens_por_pagina

    total_paginas = data.get('total_count') // itens_por_pagina # opreração de div em python
    total_paginas

    next_url = header.get('next').get('url')

In [6]:
results = requests.get(url_final)

In [122]:
resultados = pd.DataFrame(columns=['owner_type', 
                                   'owner_url'
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
resultados.head()

,owner_type,owner_urlowner_html_url,html_url,url,fork,created_at,updated_at,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme


In [121]:
def add_resultado(item):
    df = pd.DataFrame([[item['owner']['type'],
                        item['owner']['url'],
                        item['owner']['html_url'],
                        item['html_url'],
                        item['url'],
                        item['fork'],
                        item['created_at'],
                        item['updated_at'],
                        item['size'],
                        item['stargazers_count'],
                        item['language'],
                        item['has_issues'],
                        item['has_wiki'],
                        item['forks_count'],
                        item['forks'],
                        item['open_issues'],
                        item['watchers']]])

    return df    

In [ ]:
['owner']['type']
['owner']['url']
['owner']['html_url']
['html_url']
['url']
['fork']
['created_at']
['updated_at']
['size']
['stargazers_count']
['language']
['has_issues']
['has_wiki']
['forks_count']
['forks']
['license']['key']
['license']['name']
['license']['url']
['open_issues']
['watchers']

#### Alguns campos interessantes:

name
html_url
description
created_at
updated_at
pushed_at
stargazers_count
watchers_count
language
has_issues
has_projects
has_downloads
has_wiki
has_pages
forks_count
open_issues_count
license
forks
open_issues
watchers

readme??